## !!! USE sim312 !!!

conda activate sim312

In [23]:
import tsinfer
import zarr
import zarr
import tskit
import numpy as np
import bio2zarr.vcf as v2z
import msprime
import os
from IPython.display import SVG

In [14]:
# 1) Load ancestry with mutations
ts = tskit.load("trees/r3000000.trees")
print(
    f"Simulated {ts.num_samples} samples:",
    f"{ts.num_trees} trees and {ts.num_sites} sites"
)

# Drop existing sites/mutations but keep the trees:
tables = ts.dump_tables()
tables.sites.clear()
tables.mutations.clear()
ts_no_muts = tables.tree_sequence()

model = "infinite_sites"

# Re-add mutations with a nucleotide model (ancestral states will be A/C/G/T)
ts2 = msprime.sim_mutations(
    ts_no_muts, rate=2.5e-8, random_seed=42
)

AA = np.array([s.ancestral_state for s in ts2.sites()])
print(AA[:10])  # now you'll see bases like 'A','C','G','T'

np.save(f"sim-AA.npy", [s.ancestral_state for s in ts2.sites()])

Simulated 50 samples: 131 trees and 318 sites
['T' 'A' 'T' 'A' 'C' 'T' 'A' 'A' 'G' 'G']


In [16]:
# 2) Export VCF → Zarr
with open("sim.vcf", "w") as f:
    ts2.write_vcf(f, contig_id="chr1")
v2z.convert(["sim.vcf"], "sim.vcz")

Total records unknown, cannot show progress; reindex VCFs with bcftools index to fix


In [17]:
ancestral_states = np.load(f"sim-AA.npy")
vdata = tsinfer.VariantData(f"sim.vcz", ancestral_states)

In [20]:
# Run inference
inferred_ts = tsinfer.infer(vdata, progress_monitor=True, num_threads=4)

# Define your output folder and file
out_dir = "tsinfer_test"
os.makedirs(out_dir, exist_ok=True)  # Create folder if not already present

out_path = os.path.join(out_dir, "inferred.trees")

# Save the inferred tree sequence
inferred_ts.dump(out_path)

print(f"Inferred trees saved to: {out_path}")

ma-match (3/6)100%|██████████| 116/116 [00:00, 1.12kit/s]
ms-match (5/6)100%|██████████| 50.0/50.0 [00:00, 5.19kit/s]
ms-paths (6/6)100%|██████████| 50.0/50.0 [00:00, 63.9kit/s]
ms-muts  (7/6)100%|██████████| 238/238 [00:00, 48.0kit/s]
ms-xsites (8/6)100%|██████████| 79.0/79.0 [00:00, 15.5kit/s]

Inferred trees saved to: tsinfer_test/inferred.trees


In [21]:
import tskit
ts = tskit.load("tsinfer_test/inferred.trees")

print(ts)                   # summary: samples, sites, trees
print(ts.num_trees)
print(ts.num_sites)
print(ts.sequence_length)

╔════════════════════════════╗
║TreeSequence                ║
╠═══════════════╤════════════╣
║Trees          │          94║
╟───────────────┼────────────╢
║Sequence Length│     100,001║
╟───────────────┼────────────╢
║Time Units     │uncalibrated║
╟───────────────┼────────────╢
║Sample Nodes   │          50║
╟───────────────┼────────────╢
║Total Size     │    55.7 KiB║
╚═══════════════╧════════════╝
╔═══════════╤════╤═════════╤════════════╗
║Table      │Rows│Size     │Has Metadata║
╠═══════════╪════╪═════════╪════════════╣
║Edges      │ 361│ 11.3 KiB│          No║
╟───────────┼────┼─────────┼────────────╢
║Individuals│  50│  4.4 KiB│         Yes║
╟───────────┼────┼─────────┼────────────╢
║Migrations │   0│  8 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
║Mutations  │ 319│ 11.5 KiB│          No║
╟───────────┼────┼─────────┼────────────╢
║Nodes      │ 186│  8.4 KiB│         Yes║
╟───────────┼────┼─────────┼────────────╢
║Populations│   0│ 24 Bytes│          No║
╟────────

In [24]:
SVG(ts.draw_svg())